In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import Affine

import pds4_tools

from scipy import signal
from scipy.ndimage import gaussian_filter

In [2]:
def pds2numpy(file):
    data = pds4_tools.read(file)[0].data
    return data



def get_CRSinfo(file):
    structures = pds4_tools.read(file)
    label = structures.label

    ## upper left corner coordinates in meters
    ulx = label.findall(".//cart:upperleft_corner_x")[0].text
    uly = label.findall(".//cart:upperleft_corner_y")[0].text

    ## pixel resolution in m/pix
    pix_res_x = label.findall(".//cart:pixel_resolution_x")[0].text
    pix_res_y = label.findall(".//cart:pixel_resolution_y")[0].text

    ## projection info
    map_proj_name = label.findall(".//cart:map_projection_name")[0].text
    central_meridian = label.findall(".//cart:longitude_of_central_meridian")[0].text
    latitude_type = label.findall(".//cart:latitude_type")[0].text
    a_radius = label.findall(".//cart:a_axis_radius")[0].text.split(".")[0] + "000"       ## rdaius in xml in km; converting to m
    b_radius = label.findall(".//cart:b_axis_radius")[0].text.split(".")[0] + "000"
    
    proj4 = "+proj=sinu +lon_0=" + central_meridian + " +x_0=0 +y_0=0 +a="+ a_radius +" +b="+ b_radius + " +units=m +no_defs"
    transform = Affine(float(pix_res_x), 0, float(ulx), 0, -float(pix_res_y), float(uly))
    
    return proj4, transform



def correct_poweroffset(data, year):
    offset = {"1988": 4.2, "2012": 2.6, "2015": 1.6, "2017": 0, "2020": 4.7}
    corr_data = data + offset[str(year)]
    return corr_data



def correct_incangle(data, pol, incifile):
    inc = pds4_tools.read(incifile)[0].data
    
    if pol == "ocp":
        bsc_norm = 35.34 - (1.41 * inc) + (0.021 * inc**2) - (0.00011 * inc**3)
    elif pol == "scp":
        bsc_norm =  np.cos(np.deg2rad(inc))
    else:
        print("check polarization and set the right value for the variabl pol")

    data_inc_norm = np.zeros_like(data)
    bsc_norm[bsc_norm == 0.0] = np.min(bsc_norm[bsc_norm>0])     ## avoiding division by zero
    data_inc_norm = data/ bsc_norm
    
    return data_inc_norm


def calculate_cpr(ocp, scp):
    
    ocp = np.round(ocp, 6)          ## Rounding scp and ocp to 6 decimal places
    scp = np.round(scp, 6)
   
    ocp[ocp == 0.0] = np.min(ocp[ocp>0.0])             ## avoiding division by zero
    cpr = scp / ocp
    return cpr
    
    
def gaussian(data, stdev=1):
    data_filt = np.zeros_like(data)
    gaussian_filter(data, sigma=stdev, output=data_filt)
    return data_filt
    
    
def numpy2tif(outfile, data, crs, transform, nodata=9999):
    outds = rasterio.open(outfile, 'w', driver='GTiff', 
                  height = data.shape[0], 
                  width = data.shape[1], 
                  count=1, 
                  crs = crs, 
                  dtype = data.dtype,
                  transform = transform,
                  compress='lzw',
                  nodata = nodata)
    outds.write(data, 1)
    outds.close()

### 1. Reading PDS data using PDS4 tools

In [3]:
in_scp = "data2015/venus_2015_north_scp.xml"
in_ocp = "data2015/venus_2015_north_ocp.xml"
incifile = "data2015/venus_2015_north_inc.xml"
year = 2017

# out_scp = "data2015/2015_north_scp_filt.tif" 
# out_ocp = "data2015/2015_north_ocp_filt.tif"
out_cpr = "data2015/2015_north_cpr_maxwell.tif"
# out_inci = "data2015/venus_2015_north_inc.tif"

In [4]:
scp_arr = pds2numpy(in_scp)
ocp_arr = pds2numpy(in_ocp)

Processing label: data2015/venus_2015_north_scp.xml
Now processing a Array_2D_Map structure: image_object
Processing label: data2015/venus_2015_north_ocp.xml
Now processing a Array_2D_Map structure: image_object


### 2. Coordinate reference system
Affine transform arguments in rasterio
If you're coming from the matrix algebra perspective, you can ignore the constants in the affine matrix and refer to the the six paramters as a, b, c, d, e, f. This is the ordering and notation used by the affine Python library.

    a = width of a pixel
    b = row rotation (typically zero)
    c = x-coordinate of the upper-left corner of the upper-left pixel
    d = column rotation (typically zero)
    e = height of a pixel (typically negative)
    f = y-coordinate of the of the upper-left corner of the upper-left pixel


<b> Note:</b>  GDAL provides support for PDS4. <i> gdal_translate <label.xml> <outfile.tif> </i> should automatically read in PDS files. 

In [5]:
proj, transform = get_CRSinfo(in_ocp)

Processing label: data2015/venus_2015_north_ocp.xml
Now processing a Array_2D_Map structure: image_object


In [6]:
## maxwell CPR
cpr_arr = np.zeros_like(ocp_arr)
min_SNR = 1
cpr_arr[scp_arr>min_SNR] = ocp_arr[scp_arr>min_SNR] / scp_arr[scp_arr>min_SNR]
cpr_arr[cpr_arr<0.0] = 0.0
cpr_arr[cpr_arr>10.0] = 0.0
numpy2tif(out_cpr, cpr_arr, proj, transform, nodata=0)

In [7]:
cpr_filt = gaussian(cpr_arr, stdev=1)
numpy2tif("data2015/2015_north_cpr_filt_maxwell.tif", cpr_filt, proj, transform, nodata=0)

### 3. Eliminating offset in power between data from different year.
Use values from "Cal factor" column in table 2 of the User Guide (https://pds-geosciences.wustl.edu/venus/urn-nasa-pds-venus_radar_level2/document/venus_radar_maps_user_guide.pdf). 

In [8]:
ocp_offset_corr = correct_poweroffset(ocp_arr, year)
scp_offset_corr = correct_poweroffset(scp_arr, year)

### 4. Correcting for incidence angle
Incidence angle effects are removed by normalizing the data using 
1. equation in page 5 of tthe user guide for OCP data: 𝑃𝑑𝐵(𝜙) =35.34 − 1.41𝜙 + 0.021𝜙2 − 0.00011𝜙3, and 
2. cosine of the incidence angle for the SCP data.  

In [9]:
ocp_inc_norm = correct_incangle(ocp_offset_corr, "ocp", incifile)
scp_inc_norm = correct_incangle(scp_offset_corr, "scp", incifile)

Processing label: data2015/venus_2015_north_inc.xml
Now processing a Array_2D_Map structure: image_object
Processing label: data2015/venus_2015_north_inc.xml
Now processing a Array_2D_Map structure: image_object


### 5. Using a Gaussian filter to increase SNR

In [10]:
## maxwell CPR
cpr_arr = np.zeros_like(ocp_inc_norm)
min_SNR = 1
cpr_arr[scp_inc_norm>min_SNR] = ocp_inc_norm[scp_inc_norm>min_SNR] / scp_inc_norm[scp_inc_norm>min_SNR]
cpr_arr[cpr_arr<0.0] = 0.0
cpr_arr[cpr_arr>10.0] = 0.0
# numpy2tif(out_cpr, cpr_arr, proj, transform, nodata=0)

cpr_filt = gaussian(cpr_arr, stdev=1)
numpy2tif("data2015/2015_north_cpr_incnorm_filt_maxwell.tif", cpr_filt, proj, transform, nodata=0)

In [11]:
ocp_filt = gaussian(ocp_offset_corr, stdev=1)
scp_filt = gaussian(scp_offset_corr, stdev=1)

### CPR calculation (optional)
Use data that has not been normalized for incidence angle

In [12]:
cpr_filt = np.zeros_like(ocp_filt)
min_SNR = 1
cpr_filt[scp_inc_norm>min_SNR] = ocp_inc_norm[scp_inc_norm>min_SNR] / scp_inc_norm[scp_inc_norm>min_SNR]
cpr_filt[cpr_filt<0.0] = 0.0
cpr_filt[cpr_filt>10.0] = 0.0

### 6. Writing output data to file

In [13]:
numpy2tif(out_ocp, ocp_filt, proj, transform, nodata=0)
numpy2tif(out_scp, scp_filt, proj, transform, nodata=0)
numpy2tif(out_cpr, cpr_filt, proj, transform, nodata=0)

### Incidence angle files

In [13]:
inc_arr = pds2numpy(incifile)
numpy2tif(out_inci, inc_arr, proj, transform, nodata = 0.0)

Processing label: data2015/venus_2015_north_inc.xml
Now processing a Array_2D_Map structure: image_object
